In [2]:
#import the module to read from webcam 
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt


The cell below captures a video stream of the camera and displays it in the notebook. The video stream is captured using the OpenCV library. 
If we want to capture from a video file, we can use the following code:

cv.VideoCapture('project_video.mp4')

On the other hand, if we want to use the camera, we can use the following code:

cv.VideoCapture(0)


In [3]:
videoPath = "METTERE QUI IL PATH DEL VIDEO"

if videoPath == "METTERE QUI IL PATH DEL VIDEO" or videoPath == "":
    cap = cv.VideoCapture(0)
else:
    cap = cv.VideoCapture(videoPath)
    


Below, we will choose if we want to dected the frontal faces, the profile or both. We will use the frontal face cascade classifier by default.

In [4]:
haarDetectionMode = "profile"

In [5]:
haarDetectionMode = "profileAndFrontal"

In [6]:
haarDetectionMode = "frontal"

In [7]:
saveCroppedFaces = False

The function detect faces uses haar cascade to detect faces in the frame. 
There are two versions of haar cascades, one for profile faces and one for frontal faces.



In [8]:
def detectFaces(frame):

    detecedFaces = []
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    #use haar cascade to detect faces
    if haarDetectionMode == "frontal" or haarDetectionMode == "profileAndFrontal":
        faceCascadeFrontal = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
        facesFrontal = faceCascadeFrontal.detectMultiScale(gray, 1.1, 4)

        detecedFaces.extend(facesFrontal)

    elif haarDetectionMode == "profile" or haarDetectionMode == "profileAndFrontal":
        faceCascadeProfile = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_profileface.xml')
        facesProfile = faceCascadeProfile.detectMultiScale(gray, 1.1, 4)

        detecedFaces.extend(facesProfile)




    

    return detecedFaces

In [9]:
def drawRectangleFace(frame, facesCoordinates):
    #draw the rectangle around the face
    frameWithRectangle = frame.copy()
    for (x, y, w, h) in facesCoordinates:
        cv.rectangle(frameWithRectangle, (x, y), (x+w, y+h), (255, 0, 0), 2)
    return frameWithRectangle
        
        
    
        

In [10]:

def showVideo():
    ret,frame = cap.read()
    
    #flip the image
    frame = cv.flip(frame,1)

    #detect the face
    facesCoordinates = detectFaces(frame)

    #draw the rectangle around the face
    frameWithRectangle = drawRectangleFace(frame, facesCoordinates)



    #display the image
    cv.imshow('frame',frameWithRectangle)
    ret = True
    if cv.waitKey(1) & 0xFF == ord('q'):
        ret = False
        
    return ret, frame, facesCoordinates

In [11]:
def plotImage(image):
    plt.imshow(image)
    plt.show()

In [12]:
ret = True
while ret:
    
    #show the video with the face detected
    ret, frame, facesCoordinates = showVideo()

    #every two seconds, save the detected face
    if ret and len(facesCoordinates) > 0:
        for (x, y, w, h) in facesCoordinates:
            crop_img = frame[y:y+h, x:x+w]
            if saveCroppedFaces:
                cv.imwrite(".\\detectedFaces\\face" + str(x) + str(y) + ".jpg", crop_img)

                
            
    
#release the camera
cap.release()
#close all windows
cv.destroyAllWindows()

QApplication: invalid style override 'adwaita' passed, ignoring it.
	Available styles: Windows, Fusion


## The Convolutional Neural Network

We will use a convolutional neural network to detect the emotions. 
We will train our model using the FER2013 dataset. Then, using the trained model, we will detect the emotions in the faces detected in the video stream captured by the camera or the video file.


In [13]:
import  tensorflow  as  tf

from  tensorflow  import  keras

#print the number of GPU

print ( "Num GPUs Available: " ,  len ( tf.config.experimental.list_physical_devices ( 'GPU' )))

2022-11-17 18:20:29.579717: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 18:20:29.727640: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-17 18:20:29.746017: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-17 18:20:30.167686: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

Num GPUs Available:  1


2022-11-17 18:20:30.578112: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 18:20:30.582182: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 18:20:30.582275: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
